In [5]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import json

# Inicializa o driver
driver = webdriver.Chrome()
driver.get("https://sa.sinnc.app/conims/5/IDSSaude.dll")

# Injetar o script de monitoramento
script = """
(function() {
    window.actions = [];

    function getUniqueSelector(element) {
        if (element.id) {
            return `#${element.id}`;
        }

        let path = [];
        while (element && element.nodeType === Node.ELEMENT_NODE) {
            let selector = element.nodeName.toLowerCase();
            if (element.className) {
                selector += `.${Array.from(element.classList).join('.')}`;
            }

            const siblings = Array.from(element.parentNode.children).filter(el => el.nodeName === element.nodeName);
            if (siblings.length > 1) {
                selector += `:nth-of-type(${Array.prototype.indexOf.call(element.parentNode.children, element) + 1})`;
            }

            path.unshift(selector);
            element = element.parentNode;
        }
        return path.join(' > ');
    }

    document.addEventListener('click', function(event) {
        let element = event.target;
        let selector = getUniqueSelector(element);

        window.actions.push({
            type: 'click',
            selector: selector,
            timestamp: Date.now()
        });
    });

    document.addEventListener('input', function(event) {
        let element = event.target;
        let selector = getUniqueSelector(element);

        window.actions.push({
            type: 'input',
            selector: selector,
            value: element.value,
            timestamp: Date.now()
        });
    });

    document.addEventListener('keydown', function(event) {
        if (event.key === 'Escape') {
            alert('Gravação encerrada. As ações estão sendo salvas.');
            window.stopRecording = true;
        }
    });

    console.log('Monitoramento iniciado. Pressione ESC para finalizar.');
})();
"""
driver.execute_script(script)

print("Script injetado. Realize ações no navegador e pressione ESC para finalizar.")

# Aguardar até que a gravação seja encerrada
while True:
    stop_recording = driver.execute_script("return window.stopRecording || false;")
    if stop_recording:
        break

# Recuperar as ações registradas
actions = driver.execute_script("return JSON.stringify(window.actions);")

# Após registrar ações no arquivo JSON:
actions_metadata = {
    "url": driver.current_url,
    "actions": json.loads(actions),
    "timestamp": time.time(),
}
with open("actions.json", "w") as file:
    json.dump(actions_metadata, file, indent=4)

print("Ações registradas salvas em 'actions.json'")
driver.quit()

Script injetado. Realize ações no navegador e pressione ESC para finalizar.


UnexpectedAlertPresentException: Alert Text: Gravação encerrada. As ações estão sendo salvas.
Message: unexpected alert open: {Alert text : Gravação encerrada. As ações estão sendo salvas.}
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF6B2BFFB05+28789]
	(No symbol) [0x00007FF6B2B686E0]
	(No symbol) [0x00007FF6B2A0592A]
	(No symbol) [0x00007FF6B2AA01F4]
	(No symbol) [0x00007FF6B2A7F1E3]
	(No symbol) [0x00007FF6B2A4A938]
	(No symbol) [0x00007FF6B2A4BAA1]
	GetHandleVerifier [0x00007FF6B2F3933D+3410093]
	GetHandleVerifier [0x00007FF6B2F4E7DD+3497293]
	GetHandleVerifier [0x00007FF6B2F42A73+3448803]
	GetHandleVerifier [0x00007FF6B2CC7BBB+848171]
	(No symbol) [0x00007FF6B2B73C3F]
	(No symbol) [0x00007FF6B2B6F6E4]
	(No symbol) [0x00007FF6B2B6F87D]
	(No symbol) [0x00007FF6B2B5ED49]
	BaseThreadInitThunk [0x00007FFBBE467374+20]
	RtlUserThreadStart [0x00007FFBBE67CC91+33]
